## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('..\Weather_Database\Weather_Database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,La Ronge,CA,55.1001,-105.2842,41.00,60,100,6.91,overcast clouds
1,1,Albany,US,42.6001,-73.9662,53.01,87,100,7.00,moderate rain
2,2,Rikitea,PF,-23.1203,-134.9692,79.20,73,3,15.55,clear sky
3,3,Castro,BR,-24.7911,-50.0119,65.52,89,100,4.36,overcast clouds
4,4,Mar Del Plata,AR,-38.0023,-57.5575,66.20,82,20,10.36,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp).dropna() & \
                                       (city_data_df["Max Temp"] >= min_temp).dropna()]
preferred_cities_df.tail(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
685,685,San Carlos De Bariloche,AR,-41.1456,-71.3082,77.00,13,0,4.61,clear sky
686,686,San Luis,AR,-33.2950,-66.3356,61.16,41,0,10.31,clear sky
687,687,Assiniboia,CA,49.6334,-105.9845,60.80,25,89,17.27,overcast clouds
694,694,Garowe,SO,8.4054,48.4845,72.50,56,59,7.29,broken clouds
699,699,Ambon,ID,-3.6954,128.1814,77.00,88,40,5.75,scattered clouds
700,700,Nelson Bay,AU,-32.7167,152.1500,69.80,83,40,5.75,scattered clouds
704,704,Windhoek,NaN,-22.5594,17.0832,66.00,63,55,8.05,broken clouds
707,707,Baoning,CN,25.3378,110.0260,66.20,100,75,2.24,mist
709,709,Tamandare,BR,-8.7597,-35.1047,78.67,80,97,5.68,overcast clouds
711,711,Bac Lieu,VN,9.2850,105.7244,78.76,84,74,7.07,broken clouds


In [5]:
# 4a. Delete any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(inplace = True)

C:\Users\candy\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Rikitea,PF,79.20,clear sky,-23.1203,-134.9692,
3,Castro,BR,65.52,overcast clouds,-24.7911,-50.0119,
4,Mar Del Plata,AR,66.20,few clouds,-38.0023,-57.5575,
5,Esperance,AU,66.20,clear sky,-33.8667,121.9000,
9,Ribeira Grande,PT,60.80,scattered clouds,38.5167,-28.7000,
...,...,...,...,...,...,...,...
699,Ambon,ID,77.00,scattered clouds,-3.6954,128.1814,
700,Nelson Bay,AU,69.80,scattered clouds,-32.7167,152.1500,
707,Baoning,CN,66.20,mist,25.3378,110.0260,
709,Tamandare,BR,78.67,overcast clouds,-8.7597,-35.1047,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    
    #6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(inplace=True)
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Rikitea,PF,79.20,clear sky,-23.1203,-134.9692,Pension Maro'i
3,Castro,BR,65.52,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
4,Mar Del Plata,AR,66.20,few clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
5,Esperance,AU,66.20,clear sky,-33.8667,121.9000,Hospitality Esperance
9,Ribeira Grande,PT,60.80,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
...,...,...,...,...,...,...,...
694,Garowe,SO,72.50,broken clouds,8.4054,48.4845,Curubo Hotel
699,Ambon,ID,77.00,scattered clouds,-3.6954,128.1814,Swiss-Belhotel Ambon
700,Nelson Bay,AU,69.80,scattered clouds,-32.7167,152.1500,Mantra Nelson Bay
709,Tamandare,BR,78.67,overcast clouds,-8.7597,-35.1047,Pousada 100 Passos


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>WeatherDescription</dt><dd>{Weather Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

         Lat       Lng
2   -23.1203 -134.9692
3   -24.7911  -50.0119
4   -38.0023  -57.5575
5   -33.8667  121.9000
9    38.5167  -28.7000
..       ...       ...
694   8.4054   48.4845
699  -3.6954  128.1814
700 -32.7167  152.1500
709  -8.7597  -35.1047
711   9.2850  105.7244

[287 rows x 2 columns]


In [26]:
# 11a. Add a marker layer for each city to the map. 
# make hotel df heatmap for max temp
# Get the latitude and longitude.
locations = hotel_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp_pref = hotel_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable. Use list comprehension to delete any max temps that are negative because you cannot
# weigh with a negative number
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp_pref], 
                                 dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer and markers with popups.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))